<a href="https://colab.research.google.com/github/DanielKorenDataScience/ClassificationProject/blob/main/Classification_Shufersal_Presentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Classification Project - PRESENTATION**

for Shufersal on-line orders, for replacing of missing products, will predict the replacement success!

In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import display
p = print
d = display

from sklearn.preprocessing import LabelEncoder

# Get Data

In [3]:
id = 'no_diff'
prefix = 'Shufersal'

DIFF = False
RATIO = False
REL = False

REMOVE_ING = False
SPLIT_DATE = False
SPLIT_DATE2 = False
ONLY_PRICE = False

SWEETS = True
CANS = False
DETERGENTS = False

EDA = False

In [4]:
def get_data(fname, partial=True):
  data_url = 'https://raw.githubusercontent.com/DanielKorenDataScience/ClassificationProject/main/' + fname
  p(fname)

  if partial:
    # Eliminating unwanted columns - mostly descriptive columns
    df = pd.read_csv(data_url, usecols=[i for i in range(82) if not i in[0,8,10,12,14,16,18,20,24,26,27,48,50,52,54,56,58,62,64,65]])
  else:
    df = pd.read_csv(data_url)
  return df

## Sweets or Cans or Detergents

In [5]:
if SWEETS:
  df = get_data('Sweets.csv')
elif CANS:
  df = get_data('Cans.csv')
else:  
  df = get_data('Detergents.csv')
df.head(1)

Sweets.csv


,HouseHoldID,LocationCityName,QuarterNumber,MonthNumber,WeekNumber,DayNumberInWeek,PeriodHoliday,DepartmentCode,CategoryCode,GroupCode,SubGroupCode,CompoundID,ParentProductNumber,BrandIndDesc,BrandName,VendorNumber,UnitOfMeasure,Price,Quantity,cntPrdAll,NetSaleAmountAll,NetSaleNoVatPrdLast3Months,QuantSale,cntMonths,DiscountPerUnit,isPromo,Energy,Pachmema,Natran,Helbon,Shuman,ShumanRavuy,ShumanTrans,Colastrol,SivimTzunatim,CafiotSucar,DepartmentCodeSubs,CategoryCodeSubs,GroupCodeSubs,SubGroupCodeSubs,CompoundIDSubs,ProductNumberSubs,BrandIndDescSuss,BrandNameSubs,VendorNumberSubs,UnitOfMeasureSubs,PriceSubs,NetSaleNoVatPrdSubs,QuantSaleSubs,cntMonthsSubs,DiscountPerUnitSubs,EnergySubs,PachmemaSubs,NatranSubs,HelbonSubs,ShumanSubs,ShumanRavuySubs,ShumanTransSubs,ColastrolSubs,SivimTzunatimSubs,CafiotSucarSusb,CorrelateItemStatusCode
0,1,ראשון לציון,202101,202102,202108,1,NotHoliday,10,6,700,3,-8000500249352,8000500249352,קינדר,לא מותג פרטי,914547,0.205,18.9,4,22,800.63,57747.16,0.8,16,NaN,noPromo,568.0,49.5,107.0,8.6,37.8,17.3,0.5,15.0,NaN,NaN,10,6,700,6,-8000500217740,8000500217740,קינדר,לא מותג פרטי,914547,0.086,9.9,46128.25,0.8,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


# EDA

In [6]:
import seaborn as sns
try:
  from bidi import algorithm as bidialg
except:
  !pip install python-bidi
  from bidi import algorithm as bidialg

In [7]:
if EDA:
  plt.figure(figsize=(8,5))
  #ax=sns.histplot(df.CorrelateItemStatusCode.astype('int'), kde=False)
  ax=sns.countplot(x=df.CorrelateItemStatusCode)
  ax.set(title='Distribution of approvals')
  # label each bar in histogram
  for pa in ax.patches:
      height = pa.get_height() # get the height of each bar
      # adding text to each bar
      ax.text(x = pa.get_x()+(pa.get_width()/2), # x-coordinate position of data label, padded to be in the middle of the bar
      y = height+0.2, # y-coordinate position of data label, padded 0.2 above bar
      s = '{:.0f}'.format(height), # data label, formatted to ignore decimals
      ha = 'center') # sets horizontal alignment (ha) to center
  #plt.xticks(df.CorrelateItemStatusCode.astype('int').unique())
  plt.show()

In [8]:
if EDA:
  df_cp = df.copy()
  df_origin = get_data('Sweets.csv', partial=False)

In [9]:
if EDA:
  df_cp['Refused'] = np.where(df_cp.CorrelateItemStatusCode==0, 1, 0)
  df_cp = df_cp.rename(columns={'CorrelateItemStatusCode': 'Accepted'})
  df_plot = df_cp.groupby(['WeekNumber'])[['Accepted','Refused']].sum().reset_index()
  df_plot.set_index('WeekNumber').plot(kind='bar', stacked=True, figsize=(8,5))
  plt.show()

In [10]:
if EDA:
  plt.figure(figsize=(15,30))
  df_city = df_origin.groupby('LocationCityName')['CorrelateItemStatusCode'].mean().reset_index().sort_values('CorrelateItemStatusCode')
  sns.barplot( y=df_origin.LocationCityName.apply( bidialg.get_display ), x=df_origin['CorrelateItemStatusCode'], order=df_city.LocationCityName.apply( bidialg.get_display ))
  plt.show()

In [11]:
if EDA:
  plt.figure(figsize=(15,10)) 
  df_group = df_origin.groupby('GroupName')['CorrelateItemStatusCode'].mean().reset_index().sort_values('CorrelateItemStatusCode')
  sns.barplot( y=df_origin.GroupName.apply( bidialg.get_display ), x=df_origin['CorrelateItemStatusCode'], order=df_group.GroupName.apply( bidialg.get_display ))
  plt.show()

## Acceptance of Subs from different group

In [12]:
if EDA:
  plt.figure(figsize=(20,10))
  df_groupmatrix = df_origin[['GroupName', 'GroupNameSubs','CorrelateItemStatusCode']].copy()
  df_groupmatrix['grpsubsnew'] = np.where(df_groupmatrix.GroupNameSubs==df_groupmatrix.GroupName,df_groupmatrix.GroupNameSubs,'Other')
  sns.barplot(x=df_groupmatrix.GroupName.apply( bidialg.get_display ), y=df_groupmatrix.CorrelateItemStatusCode, hue=df_groupmatrix.grpsubsnew.apply( bidialg.get_display ))
  plt.show()

## Correlation - Price is negative, isPromo is positive

In [13]:
if EDA:
  plt.figure(figsize=(15,20))
  df_cor = df.corr()['CorrelateItemStatusCode'][:]
  df_cor_ = pd.DataFrame(df_cor).sort_values('CorrelateItemStatusCode')
  df_cor_ = df_cor_[df_cor_.index != 'CorrelateItemStatusCode']
  sns.barplot( y=df_cor_.index, x=df_cor_['CorrelateItemStatusCode'])
  plt.show()

# Various Data Sets - Functions

## Calc diff vector

In [14]:
def prepare_diff(df):
  id = ''
  for col in ['DepartmentCode', 'CategoryCode', 'GroupCode', #'SubGroupCode',
       'CompoundID', 'ParentProductNumber', 'BrandIndDesc',
       'VendorNumber', 'UnitOfMeasure', 'Price', 'NetSaleNoVatPrdLast3Months', 'Energy', 'Pachmema',
       'Natran', 'Helbon', 'Shuman', 'ShumanRavuy', 'ShumanTrans', 'Colastrol',
       'SivimTzunatim', 'CafiotSucar', 'QuantSale', 'cntMonths', 'DiscountPerUnit']:
    df[col + 'Diff'] = df[col] - df[col + 'Subs']

  for col in ['DepartmentCode', 'CategoryCode', 'GroupCode', #'SubGroupCode',
       'CompoundID', 'ParentProductNumber', 'VendorNumber', 'BrandIndDesc']:
    df[col + 'Diff'] = (df[col + 'Diff'] != 0)    # tested both ==0 and !=0 (the result is 1 for one and 0 for the other - no change in the model results)
    df[col + 'Diff'] = enc.fit_transform(df[col + 'Diff'])

  return df

## Calc Ratio vector - no change in results

note: changes affects only columns that are not categorical because those we change to 0/1 in the 2nd step below

In [15]:
def prepare_ratio(df):
  id = 'ratio'
  for col in ['DepartmentCode', 'CategoryCode', 'GroupCode', #'SubGroupCode',
       'CompoundID', 'ParentProductNumber', 'BrandIndDesc',
       'VendorNumber', 'UnitOfMeasure', 'Price', 'NetSaleNoVatPrdLast3Months', 'Energy', 'Pachmema',
       'Natran', 'Helbon', 'Shuman', 'ShumanRavuy', 'ShumanTrans', 'Colastrol',
       'SivimTzunatim', 'CafiotSucar', 'QuantSale', 'cntMonths', 'DiscountPerUnit']:
    df[col + 'Ratio'] = df.apply(lambda row: 999 if row[col + 'Subs'] == 0 else row[col] / row[col + 'Subs'], axis=1)

  for col in ['DepartmentCode', 'CategoryCode', 'GroupCode', #'SubGroupCode',
       'CompoundID', 'ParentProductNumber', 'VendorNumber', 'BrandIndDesc']:
    df[col + 'Ratio'] = (df[col + 'Ratio'] != 1.0)
    df[col + 'Ratio'] = enc.fit_transform(df[col + 'Ratio'])

  return df

## Calc Relative Diff - no change in results

note: changes affects only columns that are not categorical because those we change to 0/1 in the 2nd step below

In [16]:
def prepare_rel(df):
  id = 'rel_diff'

  for col in ['DepartmentCode', 'CategoryCode', 'GroupCode', #'SubGroupCode',
       'CompoundID', 'ParentProductNumber', 'BrandIndDesc',
       'VendorNumber', 'UnitOfMeasure', 'Price', 'NetSaleNoVatPrdLast3Months', 'Energy', 'Pachmema',
       'Natran', 'Helbon', 'Shuman', 'ShumanRavuy', 'ShumanTrans', 'Colastrol',
       'SivimTzunatim', 'CafiotSucar', 'QuantSale', 'cntMonths', 'DiscountPerUnit']:
    df[col + 'RelDiff'] = df.apply(lambda row: 1 if row[col] == 0 else (row[col] - row[col + 'Subs']) / row[col], axis=1)

  for col in ['DepartmentCode', 'CategoryCode', 'GroupCode', #'SubGroupCode',
       'CompoundID', 'ParentProductNumber', 'VendorNumber', 'BrandIndDesc']:
    df[col + 'RelDiff'] = (df[col + 'RelDiff'] != 0.0)
    df[col + 'RelDiff'] = enc.fit_transform(df[col + 'RelDiff'])

  return df    

## Models - remove ingredients - no change in results

In [17]:
def do_remove_ing(df):
  global X_train, X_test, y_train, y_test

  id = 'remove_ingredients'

  try:
    del X_train, X_test, y_train, y_test
  except:
    pass
  
  cols = ['CorrelateItemStatusCode']
  for col in ['Energy', 'Pachmema', 'Natran', 'Helbon', 'Shuman', 'ShumanRavuy', 'ShumanTrans', 'Colastrol',
       'SivimTzunatim', 'CafiotSucar']:
      cols.append(col)
      cols.append(col + 'Subs')
      cols.append(col + 'Diff')

  X = df.drop(cols, axis=1)
  y = df.CorrelateItemStatusCode.astype(int)

  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=0)
#  p(df.shape)
#  p(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## Models - split based on date - major change (precision went down from 84% to 78%)

In [18]:
def do_split_date(df):
  global X_train, X_test, y_train, y_test
 
  id = 'split_based_on_date'

  mask = df.WeekNumber > 202108
  try:
    del X_train, X_test, y_train, y_test
  except:
    pass

  df_train = df[~mask]
  X_train = df_train.drop('CorrelateItemStatusCode', axis=1)
  y_train = df_train.CorrelateItemStatusCode.astype(int)

  df_test = df[mask]
  X_test = df_test.drop('CorrelateItemStatusCode', axis=1)
  y_test = df_test.CorrelateItemStatusCode.astype(int)

  #Variant - remove the date related columns - must be run after the SPLIT_DATE run because it prepares the X_train
  if SPLIT_DATE2:
    id = 'split_based_on_date_without_date_features_and_order_info'

    cols = ['QuarterNumber', 'MonthNumber', 'WeekNumber', 'DayNumberInWeek', 
            'PeriodHoliday_0', 'PeriodHoliday_1',
            'Quantity', 'cntPrdAll', 'NetSaleAmountAll']
    X_train = X_train.drop(cols, axis=1)
    X_test = X_test.drop(cols, axis=1)
#    p(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

#  p(df.shape)
#  p(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## Models - remove all features execpt of prices

In [19]:
def do_only_price(df):
  global X_train, X_test, y_train, y_test

  id = 'only_prices'
  try:
    del X_train, X_test, y_train, y_test
  except:
    pass
  
  X = df[['Price', 'PriceSubs', 'PriceDiff']]
  y = df.CorrelateItemStatusCode.astype(int)

  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=0)
#  p(df.shape)
#  p(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Prepare Data

In [20]:
def prepare(df):
  ## Fix Typo mistake
  df.rename(columns={'OrderNumbert': 'OrderNumber','BrandIndDescSuss': 'BrandIndDescSubs', 'ProductNumberSubs': 'ParentProductNumberSubs', \
                    'NetSaleNoVatPrdSubs': 'NetSaleNoVatPrdLast3MonthsSubs', 'CafiotSucarSusb': 'CafiotSucarSubs'}, inplace=True)
  
  ##Eliminating nulls
  df.fillna({'QuantSale': 1, 'QuantSaleSubs': 1}, inplace=True)
  df.fillna(0, inplace=True)

  ## Encoding categories
  enc = LabelEncoder()
  enc.fit(['PromoSusbDiffPromo', 'PromoSusbSamePromo', 'noPromo'])
  df.isPromo = enc.transform(df.isPromo)

  df['LocationCityName'] = df.LocationCityName.astype('str')
  df['BrandIndDesc'] = df.BrandIndDesc.astype('str')
  df['BrandIndDescSubs'] = df.BrandIndDescSubs.astype('str')
  for col in ['LocationCityName', 'PeriodHoliday', 'BrandIndDesc', 'BrandName', 'BrandIndDescSubs', 'BrandNameSubs']:
    df[col] = enc.fit_transform(df[col])

  ## Combine 'GroupCode' and 'SubGroupCode' into one column
  df.GroupCode = df.GroupCode * 10 + df.SubGroupCode
  df.GroupCodeSubs = df.GroupCodeSubs * 10 + df.SubGroupCodeSubs

  ## Use Dummies
  ## Minimal improvement - FalseNegative (0,0) improved from 1338 to 1328
  #df = pd.get_dummies(df, columns=['isPromo', 'PeriodHoliday'])

  if DIFF:
    df = prepare_diff(df)
  elif RATIO:
    df = prepare_ratio(df)
  elif REL:
    df = prepare_rel(df)
  return df    

df = prepare(df)    

# Models


## Models - process_model code

In [21]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import math

pd.set_option("precision", 2)
df_result = pd.DataFrame()

In [22]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

X = df.drop('CorrelateItemStatusCode', axis=1)
y = df.CorrelateItemStatusCode.astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=0)

if REMOVE_ING:
  do_remove_ing(df)
elif SPLIT_DATE or SPLIT_DATE2:
  do_split_date(df)
elif ONLY_PRICE:  
  do_only_price(df)

In [23]:
def report(clf, X, y, note, test, data_type='Test'):
  global df_result
  name = type(clf).__name__ + ': ' + data_type
  acc = accuracy_score(y_true=y, y_pred=clf.predict(X))
  index = [(name, clf.classes_[0]), (name, clf.classes_[1])]
  cm = pd.DataFrame(confusion_matrix(y_true=y, y_pred=clf.predict(X)),
                    index=clf.classes_,
                    columns=clf.classes_)
  cm.reset_index(drop=True, inplace=True)
  rep = classification_report(y_true=y, y_pred=clf.predict(X), zero_division=1, output_dict=True)
  rep =  pd.DataFrame(rep)
  rep = rep.T
  rep['accuracy'] = acc
  rep.support = rep.support.astype(int)
  cols = list(rep.columns[:3]) + ['accuracy', 'support']
  rep = rep[cols]
  rep['precision_avg'] = rep.precision['macro avg']
  rep['recall_avg'] = rep.recall['macro avg']
  rep['f1-score_avg'] = rep['f1-score']['macro avg']
  rep.drop(['accuracy', 'macro avg', 'weighted avg'], axis=0, inplace=True)
  rep.reset_index(drop=True, inplace=True)
  df = cm.join(rep)
  df['name'] = name
  df['note'] = note
  df.loc[1, 'note'] = ''
  if test:  # do not add to database
    d(df)
  else:    
    df.reset_index(inplace=True)
    df_result.reset_index(inplace=True)
    df_result = df_result.append(df, ignore_index=True)
    df_result.set_index(['name', 'index'], inplace=True)
  return df_result

In [24]:
#ROC and AUC
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

def get_roc_and_auc(model,X_test,y_test):
    ns_probs = [0 for _ in range(len(y_test))]
#    lr_probs = clf_voting.predict_proba(X_test)[:,1]
    lr_probs = model.predict_proba(X_test)[:,1]
    ns_auc = roc_auc_score(y_test, ns_probs)
    lr_auc = roc_auc_score(y_test, lr_probs)
    # summarize scores
    print('\nNo Skill: ROC AUC=%.3f' % (ns_auc))
    print('model: ROC AUC=%.3f' % (lr_auc))
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
    lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
    # plot the roc curve for the model
    fig = plt.figure(figsize=(5,5))
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    plt.plot(lr_fpr, lr_tpr, marker='.', label='model')
    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(type(model).__name__ )
    # show the legend
    plt.legend()
    # show the plot
    plt.show()

In [25]:
def showFeatureImportance(model, X, threshold):
  if type(model).__name__ == 'LogisticRegression':  # no features importance data
    w0 = model.intercept_[0]
    w = model.coef_[0]
    fx_imp = pd.Series(w, index=X.columns)
  else:
    fx_imp = pd.Series(model.feature_importances_, index=X.columns)
  fx_imp /= fx_imp.sum()
  fx_imp = fx_imp.sort_values()
  fx_imp = fx_imp[fx_imp > threshold]
  fig = plt.figure(figsize=(5,5))
  plt.xlim(0, 1)
  fx_imp.plot(kind='barh', title=type(model).__name__ )

In [26]:
def process_model(model, X, y, X_test, y_test, plot=False, threshold=0, note='', test=False):
  p('running: ', type(model).__name__, X.shape)
  model.fit(X, y)
  report(model, X, y, note, test, 'Train')
  report(model, X_test, y_test, note, test)
  if plot:
    showFeatureImportance(model, X, threshold)
    get_roc_and_auc(model,X_test,y_test)    
  return df_result 

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingRegressor, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
try:
  import catboost as ctb
except:
  !pip install catboost
  !pip install xgboost
  !pip install lightgbm
import catboost as ctb  
import xgboost as xgb  
import lightgbm as lgb

     |████████████████████████████████| 67.3MB 64kB/s 



## Models - Run process_model

In [29]:
if False:
#  for lgb              
  param_dist = {"max_depth": [25,50, 75],
                "learning_rate" : [0.01,0.05,0.1],
                "num_leaves": [300,900,1200],
                "n_estimators": [200]
              }              

In [66]:
ONLY_RF = True
df_result = pd.DataFrame()

if not ONLY_RF:
  process_model(LogisticRegression(max_iter=1000), X_train, y_train, X_test, y_test, note='max_iter=1000', threshold=0.0000005)  # did not fit, return accuracy=0, so we increased max_iter from default 100
  process_model(LogisticRegression(C=100, max_iter=1000, fit_intercept=False, warm_start=True), X_train, y_train, X_test, y_test, note='max_iter=1000, C=100, fit_intercept=False, warm_start=True', threshold=0.0000005, plot=False)

  clf_GB = GradientBoostingClassifier(n_estimators=100)
  process_model(clf_GB, X_train, y_train, X_test, y_test, plot=False, threshold=0.015, note='n_estimators=100')
  clf_GB = GradientBoostingClassifier(ccp_alpha=0.0001, n_estimators=200, random_state=1234)
  process_model(clf_GB, X_train, y_train, X_test, y_test, plot=False, threshold=0.015, note='ccp_alpha=0.0001, n_estimators=200, random_state=1234')

  clf_AB = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5), n_estimators=100, learning_rate=0.01)
  process_model(clf_AB, X_train, y_train, X_test, y_test)

  process_model(KNeighborsClassifier(n_neighbors=39), X_train, y_train, X_test, y_test)

  if not REL: #ERROR: Classification metrics can't handle a mix of binary and continuous targets
    process_model(VotingClassifier(estimators=[('RF', clf_RF), ('GB', clf_GB), ('AB', clf_AB)], voting='soft'), X_train, y_train, X_test, y_test, note="('RF', clf_RF), ('GB', clf_GB), ('AB', clf_AB')")

clf_RF = RandomForestClassifier(n_estimators=200, criterion='entropy', max_depth=None, min_samples_split=20, ccp_alpha=0.0003, random_state=73)
#process_model(clf_RF, X_train, y_train, X_test, y_test, plot=True, threshold=0.015, note="n_estimators=200, criterion='entropy', max_depth=None, min_samples_split=20, ccp_alpha=0.0003, random_state=73")

id = 'take8'
model_CBC = ctb.CatBoostClassifier(logging_level='Silent', random_state=73)#, depth=None, iterations=None, learning_rate=0.1, l2_leaf_reg=5) #10
process_model(model_CBC, X_train, y_train, X_test, y_test, plot=False, threshold=0.012, note="random_state=73")#, iterations=None depth=None, learning_rate=0.1, l2_leaf_reg=5")


#model = xgb.XGBClassifier(random_state=73)
#process_model(model, X_train, y_train, X_test, y_test, plot=False, threshold=0.012)

model = lgb.LGBMClassifier(silent=False, random_state=73)#, max_depth=None, n_estimators=100, learning_rate=0.05)
process_model(model, X_train, y_train, X_test, y_test, plot=False, threshold=0.012, note="random_state=73")#, n_estimators=100 max_depth=None, random_state=73, learning_rate=0.05")

d(df_result)

fname = prefix + f'_result_{id}.csv'
df_result.to_csv(fname, index = True)
p(id)

running:  CatBoostClassifier (38122, 61)
running:  LGBMClassifier (38122, 61)


0        1  precision  recall  \
name                      index                                        
CatBoostClassifier: Train 0      10150.0   2499.0       0.88    0.80   
                          1       1340.0  24133.0       0.91    0.95   
CatBoostClassifier: Test  0       4136.0   1283.0       0.83    0.76   
                          1        855.0  10064.0       0.89    0.92   
LGBMClassifier: Train     0       9662.0   2987.0       0.85    0.76   
                          1       1676.0  23797.0       0.89    0.93   
LGBMClassifier: Test      0       4035.0   1384.0       0.82    0.74   
                          1        862.0  10057.0       0.88    0.92   

                                 f1-score  accuracy  support  precision_avg  \
name                      index                                               
CatBoostClassifier: Train 0          0.84      0.90  12649.0           0.89   
                          1          0.93      0.90  25473.0           0.89   
CatBoostClassifier: Test  0          0.79      0.87   5419.0           0.86   
                          1          0.90      0.87  10919.0           0.86   
LGBMClassifier: Train     0          0.81      0.88  12649.0           0.87   
                          1          0.91      0.88  25473.0           0.87   
LGBMClassifier: Test      0          0.78      0.86   5419.0           0.85   
                          1          0.90      0.86  10919.0           0.85   

                                 recall_avg  f1-score_avg             note  
name                      index                                             
CatBoostClassifier: Train 0            0.87          0.88  random_state=73  
                          1            0.87          0.88                   
CatBoostClassifier: Test  0            0.84          0.85  random_state=73  
                          1            0.84          0.85                   
LGBMClassifier: Train     0            0.85          0.86  random_state=73  
                          1            0.85          0.86                   
LGBMClassifier: Test      0            0.83          0.84  random_state=73  
                          1            0.83          0.84

take8


## Models - compare results

1. Diff
2. Diff with remove ingredients columns
3. Diff with split based on date
3. Diff with split based on date (without date features and order info) 
4. Ratio
5. RelDiff - relative difference
6. OnlyPrices - Diff with only 3 features( Price, PriceSubs, PriceDiff)
8. No Diff (no comparison between master and subs)

In [31]:
def get_raw(fname, subdir=''):
  data_url = 'https://raw.githubusercontent.com/DanielKorenDataScience/ClassificationProject/main/' + subdir + fname
  return pd.read_csv(data_url)

In [32]:
def read_results(id='', disp=True, full=False):
  fname = prefix + f'_result_{id}.csv'
  p(fname)
  try:
    df = pd.read_csv(fname)
  except:
    try:
      df = get_raw(fname, 'Results/')
#      df = pd.read_csv(fname)
    except:
      p('No data')
      return
  df.set_index(['name', 'index'], inplace=True)
  if not full:
    df = df.loc[['RandomForestClassifier: Train', 'RandomForestClassifier: Test']]#.copy()
  if disp:
    pd.set_option("max_column", 120)
    d(df)
  return df

In [33]:
p(id)
read_results()
read_results('remove_ingredients')
read_results('split_based_on_date')
read_results('split_based_on_date_without_date_features_and_order_info')
read_results('ratio')
read_results('rel_diff')
read_results('only_prices')
read_results('no_diff')
p()

take7
Shufersal_result_.csv


0        1  precision  recall  \
name                          index                                       
RandomForestClassifier: Train 0      9267.0   3382.0       0.86    0.73   
                              1      1546.0  23927.0       0.88    0.94   
RandomForestClassifier: Test  0      3804.0   1615.0       0.81    0.70   
                              1       879.0  10040.0       0.86    0.92   

                                     f1-score  accuracy  support  \
name                          index                                
RandomForestClassifier: Train 0          0.79      0.87  12649.0   
                              1          0.91      0.87  25473.0   
RandomForestClassifier: Test  0          0.75      0.85   5419.0   
                              1          0.89      0.85  10919.0   

                                     precision_avg  recall_avg  f1-score_avg  \
name                          index                                            
RandomForestClassifier: Train 0               0.87        0.84          0.85   
                              1               0.87        0.84          0.85   
RandomForestClassifier: Test  0               0.84        0.81          0.82   
                              1               0.84        0.81          0.82   

                                                                                  note  
name                          index                                                     
RandomForestClassifier: Train 0      n_estimators=200, criterion='entropy', max_dep...  
                              1                                                    NaN  
RandomForestClassifier: Test  0      n_estimators=200, criterion='entropy', max_dep...  
                              1                                                    NaN

Shufersal_result_remove_ingredients.csv


0        1  precision  recall  \
name                          index                                       
RandomForestClassifier: Train 0      9116.0   3533.0       0.86    0.72   
                              1      1512.0  23961.0       0.87    0.94   
RandomForestClassifier: Test  0      3767.0   1652.0       0.82    0.70   
                              1       828.0  10091.0       0.86    0.92   

                                     f1-score  accuracy  support  \
name                          index                                
RandomForestClassifier: Train 0          0.78      0.87  12649.0   
                              1          0.90      0.87  25473.0   
RandomForestClassifier: Test  0          0.75      0.85   5419.0   
                              1          0.89      0.85  10919.0   

                                     precision_avg  recall_avg  f1-score_avg  \
name                          index                                            
RandomForestClassifier: Train 0               0.86        0.83          0.84   
                              1               0.86        0.83          0.84   
RandomForestClassifier: Test  0               0.84        0.81          0.82   
                              1               0.84        0.81          0.82   

                                                                                  note  
name                          index                                                     
RandomForestClassifier: Train 0      n_estimators=200, criterion='entropy', max_dep...  
                              1                                                    NaN  
RandomForestClassifier: Test  0      n_estimators=200, criterion='entropy', max_dep...  
                              1                                                    NaN

Shufersal_result_split_based_on_date.csv


...

0        1  precision  recall  \
name                          index                                       
RandomForestClassifier: Train 0      9280.0   3369.0       0.86    0.73   
                              1      1567.0  23906.0       0.88    0.94   
RandomForestClassifier: Test  0      3829.0   1590.0       0.81    0.71   
                              1       899.0  10020.0       0.86    0.92   

                                     f1-score  accuracy  support  \
name                          index                                
RandomForestClassifier: Train 0          0.79      0.87  12649.0   
                              1          0.91      0.87  25473.0   
RandomForestClassifier: Test  0          0.75      0.85   5419.0   
                              1          0.89      0.85  10919.0   

                                     precision_avg  recall_avg  f1-score_avg  \
name                          index                                            
RandomForestClassifier: Train 0               0.87        0.84          0.85   
                              1               0.87        0.84          0.85   
RandomForestClassifier: Test  0               0.84        0.81          0.82   
                              1               0.84        0.81          0.82   

                                                                                  note  
name                          index                                                     
RandomForestClassifier: Train 0      n_estimators=200, criterion='entropy', max_dep...  
                              1                                                    NaN  
RandomForestClassifier: Test  0      n_estimators=200, criterion='entropy', max_dep...  
                              1                                                    NaN

Shufersal_result_only_prices.csv


0        1  precision  recall  \
name                          index                                       
RandomForestClassifier: Train 0      8819.0   3830.0       0.77    0.70   
                              1      2709.0  22764.0       0.86    0.89   
RandomForestClassifier: Test  0      3786.0   1633.0       0.75    0.70   
                              1      1234.0   9685.0       0.86    0.89   

                                     f1-score  accuracy  support  \
name                          index                                
RandomForestClassifier: Train 0          0.73      0.83  12649.0   
                              1          0.87      0.83  25473.0   
RandomForestClassifier: Test  0          0.73      0.82   5419.0   
                              1          0.87      0.82  10919.0   

                                     precision_avg  recall_avg  f1-score_avg  \
name                          index                                            
RandomForestClassifier: Train 0               0.81        0.80           0.8   
                              1               0.81        0.80           0.8   
RandomForestClassifier: Test  0               0.80        0.79           0.8   
                              1               0.80        0.79           0.8   

                                                                                  note  
name                          index                                                     
RandomForestClassifier: Train 0      n_estimators=200, criterion='entropy', max_dep...  
                              1                                                    NaN  
RandomForestClassifier: Test  0      n_estimators=200, criterion='entropy', max_dep...  
                              1                                                    NaN

Shufersal_result_no_diff.csv


0        1  precision  recall  \
name                          index                                       
RandomForestClassifier: Train 0      9074.0   3575.0       0.87    0.72   
                              1      1353.0  24120.0       0.87    0.95   
RandomForestClassifier: Test  0      3657.0   1762.0       0.83    0.67   
                              1       770.0  10149.0       0.85    0.93   

                                     f1-score  accuracy  support  \
name                          index                                
RandomForestClassifier: Train 0          0.79      0.87  12649.0   
                              1          0.91      0.87  25473.0   
RandomForestClassifier: Test  0          0.74      0.85   5419.0   
                              1          0.89      0.85  10919.0   

                                     precision_avg  recall_avg  f1-score_avg  \
name                          index                                            
RandomForestClassifier: Train 0               0.87        0.83          0.85   
                              1               0.87        0.83          0.85   
RandomForestClassifier: Test  0               0.84        0.80          0.82   
                              1               0.84        0.80          0.82   

                                                                                  note  
name                          index                                                     
RandomForestClassifier: Train 0      n_estimators=200, criterion='entropy', max_dep...  
                              1                                                    NaN  
RandomForestClassifier: Test  0      n_estimators=200, criterion='entropy', max_dep...  
                              1                                                    NaN

#Baseline Model

In [34]:
def returnPrec(model,products):
            productCols=['ParentProductNumber','ParentProductNumberSubs']
            val=model[(model[productCols[0]]==products[0]) & (model[productCols[1]]==products[1])]['avgApproval']
            if val.empty ==False:
                #display(val)
                return val.iloc[0]
            #print(val)
            return -1

In [35]:
class BaselineModel:
    def __init__(self, classes_):
        self.classes_ = classes_
    def fit(self,X_, y_):
        #catalogCols=['ParentProductNumber','CompoundID','SubGroupCodeNew','GroupCode']
        cols=['ParentProductNumber','ParentProductNumberSubs']
        group=pd.concat([X_[cols],y_],axis=1).apply(lambda x:x.astype(np.int64))\
            .groupby(cols)\
            .agg({'CorrelateItemStatusCode':[('avgApproval','mean'),('countRows','count')]}).reset_index()
        group.columns=group.columns.droplevel(0)
        group.columns.values[0]=cols[0]
        group.columns.values[1]=cols[1]
        group=group[group['countRows']>=5]
        #display(group.head())
        #display(len(group))
        self.model=group 
        return self
    
    def predict(self,X_): 
        #print(type(X_))
        y_predPrec = X_.apply(lambda x: returnPrec( model=self.model,products=[x['ParentProductNumber'],
                                                                                 x['ParentProductNumberSubs']]),axis=1)
        #display(y_predPrec)
        y_pred=np.where(y_predPrec>0.5,1,0)
        return y_pred
    def predict_proba(self,X_):
        y_predPrec = X_.apply(lambda x: returnPrec(model=self.model, products=[x['ParentProductNumber'],
                                                                                 x['ParentProductNumberSubs']]),axis=1)
        return y_predPrec

In [36]:
BS = False
if BS:
  baseline=BaselineModel(df.CorrelateItemStatusCode.unique())
  baseline.fit(X_train,y_train)
  y_pred=baseline.predict(X_test)

##Example baseline model

In [39]:
if BS:
  prd, prdSubs==161631, 72909799
  p(y_train[(X_train['ParentProductNumber']==prd) & (X_train['ParentProductNumberSubs']==prdSubs)].sum()\
  /len(y_train[(X_train['ParentProductNumber']==prd) & (X_train['ParentProductNumberSubs']==prdSubs)]))
  p()
  p(baseline.predict_proba(X_test[(X_test['ParentProductNumber']==prd) & (X_test['ParentProductNumberSubs']==prdSubs)]))  

0.921875

In [41]:
if BS:
  print(len(y_pred[y_pred==0]))
  print(len(y_pred[y_pred==1]))
  print(len(y_pred[y_pred==-1]))

4908
11430
0


In [42]:
if BS:
  print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.52      0.47      0.50      5419
           1       0.75      0.79      0.77     10919

    accuracy                           0.68     16338
   macro avg       0.64      0.63      0.63     16338
weighted avg       0.68      0.68      0.68     16338



In [43]:
if BS:
  process_model(baseline,  X_train, y_train, X_test, y_test)

running:  BaselineModel (38122, 61)


0        1  precision  recall  \
name                      index                                        
CatBoostClassifier: Train 0      10777.0   1872.0       0.93    0.85   
                          1        868.0  24605.0       0.93    0.97   
CatBoostClassifier: Test  0       4177.0   1242.0       0.83    0.77   
                          1        856.0  10063.0       0.89    0.92   
LGBMClassifier: Train     0       9086.0   3563.0       0.83    0.72   
                          1       1810.0  23663.0       0.87    0.93   
LGBMClassifier: Test      0       3874.0   1545.0       0.82    0.71   
                          1        867.0  10052.0       0.87    0.92   
BaselineModel: Train      0       6733.0   5916.0       0.55    0.47   
                          1      20724.0   4749.0       0.75    0.81   
BaselineModel: Test       0       2845.0   2574.0       0.52    0.47   
                          1       8585.0   2334.0       0.75    0.79   

                                 f1-score  accuracy  support  precision_avg  \
name                      index                                               
CatBoostClassifier: Train 0          0.89      0.93  12649.0           0.93   
                          1          0.95      0.93  25473.0           0.93   
CatBoostClassifier: Test  0          0.80      0.87   5419.0           0.86   
                          1          0.91      0.87  10919.0           0.86   
LGBMClassifier: Train     0          0.77      0.86  12649.0           0.85   
                          1          0.90      0.86  25473.0           0.85   
LGBMClassifier: Test      0          0.76      0.85   5419.0           0.84   
                          1          0.89      0.85  10919.0           0.84   
BaselineModel: Train      0          0.51      0.70  12649.0           0.65   
                          1          0.78      0.70  25473.0           0.65   
BaselineModel: Test       0          0.50      0.68   5419.0           0.64   
                          1          0.77      0.68  10919.0           0.64   

                                 recall_avg  f1-score_avg  \
name                      index                             
CatBoostClassifier: Train 0            0.91          0.92   
                          1            0.91          0.92   
CatBoostClassifier: Test  0            0.85          0.85   
                          1            0.85          0.85   
LGBMClassifier: Train     0            0.82          0.83   
                          1            0.82          0.83   
LGBMClassifier: Test      0            0.82          0.83   
                          1            0.82          0.83   
BaselineModel: Train      0            0.64          0.65   
                          1            0.64          0.65   
BaselineModel: Test       0            0.63          0.63   
                          1            0.63          0.63   

                                                                              note  
name                      index                                                     
CatBoostClassifier: Train 0      iterations=None depth=None, random_state=73, l...  
                          1                                                         
CatBoostClassifier: Test  0      iterations=None depth=None, random_state=73, l...  
                          1                                                         
LGBMClassifier: Train     0      n_estimators=100 max_depth=None, random_state=...  
                          1                                                         
LGBMClassifier: Test      0      n_estimators=100 max_depth=None, random_state=...  
                          1                                                         
BaselineModel: Train      0                                                         
                          1                                                         
BaselineModel: Test       0                     

# RECOMMENDATION LIST

## Method 1 - Zvi

In [44]:
model_RF = RandomForestClassifier(n_estimators=200, criterion='entropy', max_depth=None, min_samples_split=20, ccp_alpha=0.0003, random_state=73)
model_RF.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0003, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=20,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=73, verbose=0,
                       warm_start=False)

In [45]:
result = pd.DataFrame(model_RF.predict_proba(X))

result['Product'] = X.ParentProductNumber
result['ProductSubs'] = X.ParentProductNumberSubs
result['Selected'] = y
result['Predicted'] = model_RF.predict(X)
result['Correct'] = (result['Predicted'] == result['Selected'])
result.head()

,0,1,Product,ProductSubs,Selected,Predicted,Correct
0,0.26,0.74,8000500249352,8000500217740,1,1,True
1,0.10,0.90,178134,7290100852576,1,1,True
2,0.63,0.37,7610400091060,8690766156404,0,0,True
3,0.04,0.96,7290112490124,7290107649605,1,1,True
4,0.14,0.86,7296073382416,7296073237631,1,1,True


In [46]:
result_stat = result.drop([0,1], axis=1).groupby(['Product', 'ProductSubs']).agg({'Correct': [('AvgCorrect','mean'),('CountCorrect','count')]})

In [47]:
result_stat.head()

Correct             
                    AvgCorrect CountCorrect
Product ProductSubs                        
161631  161648            0.91           45
        72909799          0.88           86
        72917329          1.00            1
        72917367          1.00            2
        72917893          0.75            4

### List of Product/subs with at least 3 entries in the dataset, and the avgCorrectness of our model compared to actual

In [48]:
mask3 = (result_stat.iloc[:,1] >=3)
df_res = result_stat[mask3]
df_res.head(10)

Correct             
                      AvgCorrect CountCorrect
Product ProductSubs                          
161631  161648              0.91           45
        72909799            0.88           86
        72917893            0.75            4
        5000159407236       0.67            3
        7290112497918       0.96           24
161648  161631              0.99           71
        72909799            0.94           68
        7290112497918       0.94           31
170053  170077              0.73           15
        170275              0.92           50

### Lists of products and their subs, top 3 subs per product based on avgCorrect 

In [49]:
# the next groupby remove the columns from the df, so we trick it and add it to the index - groupby preserve the index
df_recommend = df_res.set_index(('Correct','CountCorrect'), append=True)
df_recommend.head()

Correct
                                              AvgCorrect
Product ProductSubs   (Correct, CountCorrect)           
161631  161648        45                            0.91
        72909799      86                            0.88
        72917893      4                             0.75
        5000159407236 3                             0.67
        7290112497918 24                            0.96

In [50]:
# now we can use groupby and nlargset to keep the 3 items with the highest AvgCorrect
df_recommend = pd.DataFrame(df_recommend[('Correct','AvgCorrect')].groupby('Product', group_keys=False).nlargest(3))
df_recommend.head()

Correct
                                              AvgCorrect
Product ProductSubs   (Correct, CountCorrect)           
161631  7290112497918 24                            0.96
        161648        45                            0.91
        72909799      86                            0.88
161648  161631        71                            0.99
        72909799      68                            0.94

In [51]:
df_recommend[("", "Rank")] = df_recommend.groupby(['Product'])[[('Correct', 'AvgCorrect')]].rank(method="first", ascending=False).astype('int')
df_recommend.head(3)

Correct     
                                              AvgCorrect Rank
Product ProductSubs   (Correct, CountCorrect)                
161631  7290112497918 24                            0.96    1
        161648        45                            0.91    2
        72909799      86                            0.88    3

In [52]:
df_recommend.reset_index(inplace=True)
df_recommend.set_index(['Product', 'ProductSubs'], inplace=True)
df_recommend.head()

Correct                
                      CountCorrect AvgCorrect Rank
Product ProductSubs                               
161631  7290112497918           24       0.96    1
        161648                  45       0.91    2
        72909799                86       0.88    3
161648  161631                  71       0.99    1
        72909799                68       0.94    2

## Method 2 - Itay

### Get Top Predict Substitute

In [53]:
def getBestSubstitute(model, X, numOfSubs=2 ):
        # get soft prediction
        y_pred_soft=model.predict_proba(X)
        df_proba=pd.concat([X.reset_index(drop=True),pd.DataFrame(y_pred_soft[:,1],columns=['prdictProba'])],axis=1)
        # get mean and std for each  product and Subs
        dfGroup=df_proba.groupby(['ParentProductNumber','ParentProductNumberSubs']).agg({'prdictProba':[('prdictProbaMean','mean'),
                                                                                       ('prdictProbaStd',np.std),
                                                                       ('countRows','count')]}).reset_index()
        dfGroup.columns=dfGroup.columns.droplevel(0)
        dfGroup.columns.values[0]='ParentProductNumber'
        dfGroup.columns.values[1]='ParentProductNumberSubs'
        dfGroup=dfGroup[dfGroup['countRows']>=3]
        # get rank of mean and std
        dfGroup['scoreMean']=dfGroup.groupby('ParentProductNumber')['prdictProbaMean'].apply(lambda x: (x-x.min())/(x.max()-x.min()))
        dfGroup['scoreStd']=dfGroup.groupby('ParentProductNumber')['prdictProbaStd'].apply(lambda x: 1-((x-x.min())/(x.max()-x.min())))
        dfGroup['scoreMeanAndStd']=(dfGroup['scoreMean']*0.8+dfGroup['scoreStd']*0.2).astype(np.float64)
        dfGroup=dfGroup.round({'scoreMeanAndStd':4})
        dfGroup['rankMeanAndStd']=dfGroup.groupby('ParentProductNumber')['scoreMeanAndStd'].rank(ascending=0,method='dense')
        dfGroup['rankMeanAndStd']=np.where(dfGroup['rankMeanAndStd'].isnull(),1,dfGroup['rankMeanAndStd'])
        dfGroup.sort_values(['ParentProductNumber','rankMeanAndStd'],inplace=True)
        # filter best susbs by numofSubs
        dfGroupFinal=dfGroup[(dfGroup['rankMeanAndStd']<=numOfSubs)]
        return dfGroupFinal

In [54]:
df_bestTwoSubs=getBestSubstitute(model_RF, X)
p(df_bestTwoSubs.shape)
df_bestTwoSubs.head()

(935, 9)


,ParentProductNumber,ParentProductNumberSubs,prdictProbaMean,prdictProbaStd,countRows,scoreMean,scoreStd,scoreMeanAndStd,rankMeanAndStd
0,161631,161648,0.89,0.12,45,1.00,0.96,0.99,1.0
12,161631,7290112497918,0.89,0.12,24,0.99,1.00,0.99,2.0
16,161648,72909799,0.91,0.11,68,1.00,1.00,1.00,1.0
14,161648,161631,0.89,0.14,71,0.53,0.00,0.43,2.0
35,170053,5197628,0.73,0.17,15,0.95,0.55,0.87,1.0


### get Top Actual

In [55]:
def getTopActual(df,NTop=2):
    df_groupActual=df.groupby(['ParentProductNumber','ParentProductNumberSubs']).agg({'CorrelateItemStatusCode':[('AvgApproval','mean'),
                                                                                              ('StdApproval',np.std),
                                                                                              ('cntRows','count')
                                                                                             ]}).reset_index()
    df_groupActual.columns=df_groupActual.columns.droplevel(0)
    df_groupActual.columns.values[0]='ParentProductNumber'
    df_groupActual.columns.values[1]='ParentProductNumberSubs'
    df_groupActual=df_groupActual[df_groupActual['cntRows']>=3]
    df_groupActual.sort_values(['ParentProductNumber','AvgApproval'],ascending=[True,False],inplace=True)
    df_ActualTopN=df_groupActual.groupby('ParentProductNumber').head(NTop)
    return df_ActualTopN

In [56]:
df_ActualTop3 = getTopActual(df,3)
p(df_ActualTop3.shape)
df_ActualTop3.head(8)

(1283, 5)


,ParentProductNumber,ParentProductNumberSubs,AvgApproval,StdApproval,cntRows
12,161631,7290112497918,0.92,0.28,24
0,161631,161648,0.91,0.29,45
1,161631,72909799,0.88,0.32,86
16,161648,72909799,0.94,0.24,68
30,161648,7290112497918,0.94,0.25,31
14,161648,161631,0.92,0.28,71
41,170053,7290104720536,0.91,0.30,11
35,170053,5197628,0.80,0.41,15


### compare Top Predict In Top Actual

In [57]:
def precKPredictInNActual(KToppredict=1, NTopActual=2):
    df_ActualTop=getTopActual(df,NTopActual)
    display(df_ActualTop.head(3))
    df_RecTop=getBestSubstitute(clf_RF, X,KToppredict)
    df_isInPred=df_ActualTop.merge(df_RecTop,on=['ParentProductNumber','ParentProductNumberSubs'],how='left')\
    .groupby('ParentProductNumber').agg({'prdictProbaMean':[('countSubs','count')],
                                        'ParentProductNumberSubs':[('countRowsMainPrd','count')]}).reset_index()
    df_isInPred.columns=df_isInPred.columns.droplevel(0)
    df_isInPred.columns.values[0]='ParentProductNumber'
    display(df_isInPred[df_isInPred['ParentProductNumber']==161631])
    #df_isInPred.to_clipboard()  # not working in colab
    print(len(df_isInPred[((df_isInPred['countSubs']<KToppredict) & (df_isInPred['countRowsMainPrd']>KToppredict))|
          (df_isInPred['countSubs']==0)]))
    print(len(df_isInPred[(df_isInPred['countSubs']>=KToppredict) |
          ((df_isInPred['countRowsMainPrd']<=KToppredict) &(df_isInPred['countSubs']>0))]))
    print("{:.0%}".format(len(df_isInPred[(df_isInPred['countSubs']>=KToppredict) |
          ((df_isInPred['countRowsMainPrd']<=KToppredict) &(df_isInPred['countSubs']>0))])/len(df_isInPred)))

In [58]:
precKPredictInNActual()

,ParentProductNumber,ParentProductNumberSubs,AvgApproval,StdApproval,cntRows
12,161631,7290112497918,0.92,0.28,24
0,161631,161648,0.91,0.29,45
16,161648,72909799,0.94,0.24,68


NotFittedError: ignored

In [ ]:
precKPredictInNActual(2,3)

## Compare to Itay's and Zvi's lists 

## If need to get the data from GitHub - instead of running the codes

In [ ]:
def get_data(fname, header=None):
  data_url = 'https://raw.githubusercontent.com/DanielKorenDataScience/ClassificationProject/main/' + fname
  if not header:
    df = pd.read_csv(data_url, index_col=0)
  else:
    df = pd.read_csv(data_url, index_col=0, header=header)
  return df

In [ ]:
REMOTE = True
if REMOTE:
  df_zvi = get_data('Results/zvi_recommendation_list.csv', header=[0,1])
  cols = pd.MultiIndex.from_tuples([("","Product"), ("","ProductSubs"), ("Correct","CountCorrect"), ("Correct","AvgCorrect"), ("Correct","Rank")])
  df_zvi.columns = cols
  df_zvi.columns = df_zvi.columns.droplevel()
  df_zvi.set_index(['Product', 'ProductSubs'], inplace=True)
  cols = pd.MultiIndex.from_tuples([("Correct","CountCorrect"), ("Correct","AvgCorrect"), ("Correct","Rank")])
  df_zvi.columns = cols

  df_itay = get_data('Results/itay_recommendation_list.csv')
else:
  df_zvi = df_recommend
  df_itay = df_ActualTop3.copy()

df_itay.set_index(['ParentProductNumber', 'ParentProductNumberSubs'], inplace=True)
p("Itay's list")
d(df_itay.head(6))
p('\n------------------------')
p(("Zvi's list"))
df_zvi.head(6)

 The 2 lists have same product because the have same number of rows, and all items from itay's list are in zvi's list

 > **NOTE:** looking at the lists we see that the order within each product, of the subs, is not identical (for exampel look above for product 161648 - itay's 1st and 3rd are zvi's 3rd and 1st)

In [ ]:
def check_exists(idx):
  try:
    df_recommend.loc[idx]
    return True
  except:
    p(idx)
    return False

not_found =[]
for idx in df_itay.index:
  if not check_exists(idx):
    not_found.append(idx)
  break
  
not_found

# Grid Search for hyperparameters - on best model RandonForest

It run for few hours and then colab disconnected - it was to preform 5880 checks. From the partial results (because Verbose=4) I saw that n_estimators=500, min_samples_split=6, criterion='entropy' gives better results so we implemented them and we can see that indeed precision was improved from 84% to 85% 

In [60]:
GS = False
if GS and False:
  clf_RF = RandomForestClassifier(n_estimators=64)
  clf_RF.fit(X_train, y_train)
  p(clf_RF.score(X_train, y_train))
  p(clf_RF.score(X_test, y_test))

In [64]:
def run_GS(model, param_grid, n_jobs=-1, cv=5, verbose=4):
  global clf_gs
  if verbose > 3 and n_jobs == -1:
    p('Warning: when n_jobs=-1 no partial results are displayed')
    p('Change one of the parameters - Aborted')
    return
  clf_gs = GridSearchCV(model, param_grid, cv=cv, verbose=verbose, n_jobs=n_jobs)
  clf_gs.fit(X_train, y_train)

  p('Results')
  clf_gs.estimator
  p(clf_gs.best_params_)
  p(clf_gs.best_estimator_)
  p('After')
  p(clf_gs.score(X_train, y_train))
  p(clf_gs.score(X_test, y_test))

In [ ]:
if GS or True:
  param_grid = {'depth'         : [4,6,8,10],
                 'learning_rate' : [0.01,0.03,0.04],
                  'iterations'    : [10,30,60,100,200,500],
                'l2_leaf_reg': [1,3,5],
                  'random_state': [73]
                }
  model = model_CBC
  run_GS(model, param_grid, n_jobs=-1, cv=5, verbose=3)

In [ ]:
if GS or True:
  param_grid = {'depth'         : [None, 10, 14, 20],
                 'learning_rate' : [0.04, 0.1],
 #                 'iterations'    : [500, 700],
                'l2_leaf_reg': [5, 10],
                  'random_state': [73]
                }
  model = model_CBC
  run_GS(model, param_grid, n_jobs=1, cv=2, verbose=3)

In [65]:
if GS or True:
  param_grid = {#'depth'         : [None],
                 'learning_rate' : [0.1, 0.15, 0.2],
 #                 'iterations'    : [500, 700],
 #               'l2_leaf_reg': [5, 10],
                  'random_state': [73]
                }
  model = model_CBC
  run_GS(model, param_grid, n_jobs=1, cv=2, verbose=3)

Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV] learning_rate=0.1, random_state=73 ..............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .. learning_rate=0.1, random_state=73, score=0.859, total=  16.6s
[CV] learning_rate=0.1, random_state=73 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.6s remaining:    0.0s


[CV] .. learning_rate=0.1, random_state=73, score=0.852, total=  16.5s
[CV] learning_rate=0.15, random_state=73 .............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   33.1s remaining:    0.0s


[CV] . learning_rate=0.15, random_state=73, score=0.857, total=  16.5s
[CV] learning_rate=0.15, random_state=73 .............................
[CV] . learning_rate=0.15, random_state=73, score=0.850, total=  16.5s
[CV] learning_rate=0.2, random_state=73 ..............................
[CV] .. learning_rate=0.2, random_state=73, score=0.852, total=  16.5s
[CV] learning_rate=0.2, random_state=73 ..............................
[CV] .. learning_rate=0.2, random_state=73, score=0.851, total=  16.4s


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  1.7min finished


Results
{'learning_rate': 0.1, 'random_state': 73}
After
0.928125491841981
0.8715877096339821


In [ ]:
if GS and False:
  param_grid = { 
      'max_depth' : [None, 8, 15],
      'ccp_alpha': [0.0001, 0.0003, 0.0005],
      'n_estimators': [100, 200],
  #    'max_features': ['auto', 'sqrt', 'log2', 1.0, 0.75, 0.5, 0.25],
      'max_leaf_nodes': [None, 20, 40],
      'min_samples_split': [2, 10, 20],
      'random_state': [73]
  }
  model = GradientBoostingClassifier()
  run_GS(model, param_grid, n_jobs=-1, cv=2, verbose=3)

In [ ]:
if GS:
  param_grid = { 
#      'max_depth' : [None, 8, 15],
      'ccp_alpha': [0.0008, 0.001, 0.002],
      #'n_estimators': [100],
  #    'max_features': ['auto', 'sqrt', 'log2', 1.0, 0.75, 0.5, 0.25],
#      'max_leaf_nodes': [None, 20, 40],
      'min_samples_split': [2],
      'random_state': [73]
  }
  model = GradientBoostingClassifier()
  run_GS(model, param_grid, n_jobs=-1, cv=2, verbose=3)

Results
{'ccp_alpha': 0.0005, 'max_depth': None, 'max_leaf_nodes': None, 'min_samples_split': 2, 'n_estimators': 100, 'random_state': 73}
GradientBoostingClassifier(ccp_alpha=0.0005, criterion='friedman_mse',
                           init=None, learning_rate=0.1, loss='deviance',
                           max_depth=None, max_features=None,
                           max_leaf_nodes=None, min_impurity_decrease=0.0,
                           min_impurity_split=None, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           presort='deprecated', random_state=73, subsample=1.0,
                           tol=0.0001, validation_fraction=0.1, verbose=0,
                           warm_start=False)

After

0.8204559424860357

0.764712785307751


In [ ]:
if GS:
  param_grid = { 
      'max_depth' : [None, 8, 15],
      'ccp_alpha': [0.0001, 0.0003, 0.0005],
      'n_estimators': [100, 200],
  #    'max_features': ['auto', 'sqrt', 'log2', 1.0, 0.75, 0.5, 0.25],
      'criterion' :['gini', 'entropy'], 
      'max_leaf_nodes': [None, 20, 40],
      'min_samples_split': [2, 10, 20],
      'random_state': [73]
  }
  model = RandomForestClassifier()
  run_GS(model, param_grid, n_jobs=-1, cv=2, verbose=3)

In [ ]:
if GS:
  param_grid = { 
      'max_depth' : [None, 4,5,6,7,8],
      'ccp_alpha': [0.01, 0.02, 0.03, 0.1],
  #    'n_estimators': [64, 200, 500],
  #    'max_features': ['auto', 'sqrt', 'log2', 1.0, 0.75, 0.5, 0.25],
      'criterion' :['gini', 'entropy'], 
  #   'max_leaf_nodes': [None, 4, 10, 20],
      'min_samples_split': [20, 100, 500]#[2, 4, 6, 8, 10, 12, 14]
  }
  model = RandomForestClassifier()
  run_GS(model, param_grid, n_jobs=-1, cv=2, verbose=3)

In [ ]:
if GS and False:
  param_grid = { 
      'max_depth' : [None, 4,8,10],
      'ccp_alpha': [0.005, 0.01, 0.015],
  #    'n_estimators': [64, 200, 500],
  #    'max_features': ['auto', 'sqrt', 'log2', 1.0, 0.75, 0.5, 0.25],
      'criterion' :['entropy'], 
  #   'max_leaf_nodes': [None, 4, 10, 20],
      'min_samples_split': [10, 20, 30]
  }

  model = RandomForestClassifier()
  run_GS(model, param_grid, n_jobs=-1, cv=2, verbose=3)

> output:

Fitting 5 folds for each of 144 candidates, totalling 720 fits

{'ccp_alpha': 0.01, 'criterion': 'entropy', 'max_depth': 8, 'min_samples_split': 20}

In [ ]:
if GS and False:
  param_grid = { 
      'max_depth' : [None, 4,8,10],
      'ccp_alpha': [0.005, 0.01, 0.015],
  #    'n_estimators': [64, 200, 500],
  #    'max_features': ['auto', 'sqrt', 'log2', 1.0, 0.75, 0.5, 0.25],
      'criterion' :['entropy'], 
  #   'max_leaf_nodes': [None, 4, 10, 20],
      'min_samples_split': [10, 20, 30]
  }

  model = RandomForestClassifier()
  run_GS(model, param_grid, n_jobs=-1, cv=2, verbose=3)

> Output

Fitting 5 folds for each of 36 candidates, totalling 180 fits

{'ccp_alpha': 0.005, 'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 20}

Train: 0.7725460364094224

Test: 0.7751866813563472

In [ ]:
if GS and False:
  param_grid = { 
      'max_depth' : [None],
      'ccp_alpha': [0.001, 0.003, 0.005],
      'criterion' :['entropy'], 
      'min_samples_split': [20],
      'n_estimators': [64, 200, 500],
  }
  model = RandomForestClassifier()
  run_GS(model, param_grid, n_jobs=-1, cv=2, verbose=3)

> Output

Fitting 5 folds for each of 9 candidates, totalling 45 fits

{'ccp_alpha': 0.001, 'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 20, 'n_estimators': 200}

Train: 0.8159592885997586

Test: 0.8142367486840495

In [ ]:
if GS and False:
  param_grid = { 
      'max_depth' : [None],
      'ccp_alpha': [0.0001, 0.0005, 0.0008, 0.001],
      'criterion' :['entropy'], 
      'min_samples_split': [20],
      'n_estimators': [200, 300],
  }
  model = RandomForestClassifier()
  run_GS(model, param_grid, n_jobs=-1, cv=2, verbose=3)

> Output

Fitting 5 folds for each of 8 candidates, totalling 40 fits (15 mins)

{'ccp_alpha': 0.0001, 'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 20, 'n_estimators': 200}

Train: 0.921698756623472

Test: 0.8595299302240176

In [ ]:
if GS and False:
  clf_RF = RandomForestClassifier(n_estimators=200, criterion='entropy', max_depth=None, min_samples_split=20, ccp_alpha=0.0005)
  clf_RF.fit(X_train, y_train)
  p(clf_RF.score(X_train, y_train))
  p(clf_RF.score(X_test, y_test))

In [ ]:
#{'ccp_alpha': 0.0005, 'criterion': 'gini', 'max_depth': None, 'max_leaf_nodes': None, 'min_samples_split': 2, 'n_estimators': 100, 'random_state': 73}

if GS:
  for i in range(5):
    ccp_alpha = 0.0001 + i / 10000
    clf_RF = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, ccp_alpha=ccp_alpha, random_state=73)
    p(ccp_alpha)
    clf_RF.fit(X_train, y_train)
    p(clf_RF.score(X_train, y_train))
    p(clf_RF.score(X_test, y_test))

# Cross validation

In [ ]:
CV = False
if CV:
  k = 7
  scores = cross_val_score(clf_RF, X_train, y_train, cv=k)
  p("\nscores : " + (k * "{:.3f}, ").format(*scores))
  p("mean : {:.3f} std : {:.3f} ".format(scores.mean(), scores.std()))

scores : 0.838, 0.845, 0.843, 0.850, 0.837, 0.844, 0.843, 

mean : 0.843 std : 0.004 